In [28]:
import os

In [29]:
%pwd

'c:\\Text-Summarization\\research'

In [30]:
os.chdir("../")

In [31]:
%pwd

'c:\\Text-Summarization'

In [64]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [65]:
import sys
sys.path.append("src")

In [66]:

%pip install -e src

import sys
from pathlib import Path

src_path = str((Path.cwd().parent / "src").resolve())
if src_path not in sys.path:
	sys.path.append(src_path)

from Text_Summarization.constants import *
from Text_Summarization.utils.common import read_yaml, create_dictionaries


Obtaining file:///C:/Text-Summarization/src
Note: you may need to restart the kernel to use updated packages.


ERROR: file:///C:/Text-Summarization/src does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [67]:
from pathlib import Path
CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_dictionaries([self.config.artifacts_root],verbose=True)

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_dictionaries([config.root_dir],verbose=True)

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [68]:
import os
import urllib.request as request
import zipfile
from Text_Summarization.utils.common import get_size
from Text_Summarization.logging import logging


In [69]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logging.info(f"{filename} download! with following info: \n{headers}")
        else:
            logging.info(f"Data already downloaded to {self.config.local_data_file}")

    def unzip_data(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [74]:
try:
    if not CONFIG_FILE_PATH.exists():
        raise FileNotFoundError(f"Config file not found: {CONFIG_FILE_PATH.resolve()}")

    # Remove corrupted zip file if it exists
    if os.path.exists(local_zip) and not zipfile.is_zipfile(local_zip):
        os.remove(local_zip)

    data_ingestion = DataIngestion(data_ingestion_config)

    data_ingestion.download_data()

    # Check if the downloaded file is a valid zip file before unzipping
    if not os.path.exists(local_zip):
        raise FileNotFoundError(f"Downloaded file not found: {local_zip}")
    if not zipfile.is_zipfile(local_zip):
        os.remove(local_zip)
        raise zipfile.BadZipFile(f"Downloaded file is not a valid zip file: {local_zip}")
    data_ingestion.unzip_data()
except Exception as e:
    print(f"Error: {e}")


[2025-08-23 13:01:06,220] INFO - 2040867305 - artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4535:3495A1:5733A:EB3D9:68A9687C
Accept-Ranges: bytes
Date: Sat, 23 Aug 2025 07:31:00 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10239-MAA
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1755934260.209472,VS0,VE222
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 4a3012c0beb0320640041f04620d16f33dbd7f2f
Expires: Sat, 23 Aug 2025 07:36:00 GMT
Source-Age: 0


Error: 


In [75]:
import os
print(os.path.getsize("artifacts/data_ingestion/data.zip"))

0
